In [57]:
#!pip install PyPDF2

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import io
import PyPDF2

In [2]:
from tqdm import tqdm

In [3]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import strip_tags
from gensim.models.doc2vec import TaggedDocument

from lbl2vec import Lbl2Vec
from gensim.utils import simple_preprocess
from gensim.models.doc2vec import TaggedDocument
from gensim.parsing.preprocessing import strip_tags
from sklearn.metrics import f1_score

In [4]:
from lbl2vec import Lbl2Vec, Lbl2TransformerVec
import torch

# evaluation
from sklearn.metrics import f1_score
#splitting data
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv('data_election_bridge.csv')

In [8]:
df.head()

,link,category
0,https://content.civicplus.com/api/assets/bcff9...,Safety
1,https://content.civicplus.com/api/assets/f8a03...,Zoning
2,https://content.civicplus.com/api/assets/cc288...,Safety
3,https://content.civicplus.com/api/assets/0ea69...,Zoning
4,https://content.civicplus.com/api/assets/8ec46...,Education


In [9]:
df["category"].unique()

array(['Safety', 'Zoning', 'Education', 'Infrastructure', 'Environmental'],
      dtype=object)

In [10]:
df["category"].value_counts()

category
Education         102
Safety             75
Environmental      60
Infrastructure     57
Zoning             52
Name: count, dtype: int64

In [11]:
df = df[(df["category"].isin(['Education', 'Infrastructure', 'Zoning', 'Safety', 'Environmental']))]

In [12]:
df["category"].value_counts()

category
Education         102
Safety             75
Environmental      60
Infrastructure     57
Zoning             52
Name: count, dtype: int64

In [13]:
df.shape

(346, 2)

In [15]:
texts = []
for i in tqdm(df['link']):
    response = requests.get(i)
    pdf_io_bytes = io.BytesIO(response.content)
    text_list = []
    pdf = PyPDF2.PdfReader(pdf_io_bytes)

    num_pages = len(pdf.pages)

    for page in range(num_pages):
        page_text = pdf.pages[page].extract_text()
        text_list.append(page_text)
    text = "\n".join(text_list)
    texts.append(text)

 31%|████████████████████████▋                                                       | 107/346 [03:11<03:27,  1.15it/s]unknown widths : 
[0, IndirectObject(198, 0, 2149312502672)]
unknown widths : 
[0, IndirectObject(193, 0, 2149312502672)]
unknown widths : 
[0, IndirectObject(188, 0, 2149312502672)]
unknown widths : 
[0, IndirectObject(178, 0, 2149312502672)]
unknown widths : 
[0, IndirectObject(173, 0, 2149312502672)]
unknown widths : 
[0, IndirectObject(167, 0, 2149312502672)]
unknown widths : 
[0, IndirectObject(156, 0, 2149312502672)]
unknown widths : 
[0, IndirectObject(151, 0, 2149312502672)]
unknown widths : 
[0, IndirectObject(142, 0, 2149312502672)]
 48%|██████████████████████████████████████▌                                         | 167/346 [04:50<05:58,  2.00s/it]unknown widths : 
[0, IndirectObject(816, 0, 2149297041488)]
unknown widths : 
[0, IndirectObject(811, 0, 2149297041488)]
unknown widths : 
[0, IndirectObject(806, 0, 2149297041488)]
unknown widths : 
[0, Indirect

In [16]:
len(texts)

346

In [17]:
#texts

In [18]:
df["Description"] = texts

In [19]:
df.head()

,link,category,Description
0,https://content.civicplus.com/api/assets/bcff9...,Safety,Order 1 1-21/22 \nPassage: 9 -0 on 7/19/2021 ...
1,https://content.civicplus.com/api/assets/f8a03...,Zoning,\nOrder 1 2-21/22 \nPassage: 9 -0 on 7/19/202...
2,https://content.civicplus.com/api/assets/cc288...,Safety,\n \n13432743.1 \nOrder 1 3-21/22 \nPassage...
3,https://content.civicplus.com/api/assets/0ea69...,Zoning,\n1 Order 14-21/22 \nPassage: 9-0 on 7/19/20...
4,https://content.civicplus.com/api/assets/8ec46...,Education,Order 15 -21/22 \nMotion to amend with Amendme...


In [20]:
df.tail()

,link,category,Description
341,https://content.civicplus.com/api/assets/89003...,Safety,Order 2 55-22/23 \nPassage as an emergency : 7...
342,https://content.civicplus.com/api/assets/a147d...,Education,Order 260 -22/23 \nMotion to amend to set eff...
343,https://content.civicplus.com/api/assets/a258d...,Education,Orders \Appointments \Police Chief Sauschuck ...
344,https://content.civicplus.com/api/assets/19ab9...,Education,Order 263- 22/23 \nPassage as an emergency : 7...
345,https://content.civicplus.com/api/assets/175b1...,Environmental,Order 173- 22/23 \nPassage: 9 -0 on 4/24/2023 ...


In [21]:
df["Description"][1]

' \nOrder 1 2-21/22 \nPassage: 9 -0 on 7/19/2021      Effective 7/29/2021 \n \nKATE SNYDER (MAYOR)  \nBELINDA S. RAY (1)  \nSPENCER R. THIBODEAU (2)  \nTAE Y. CHONG (3)  \nANDREW ZARRO (4)  CITY OF PORTLAND  \nIN THE CITY COUNCIL  MARK DION (5)  \nAPRIL D. FOURNIER(A/L)  \nPIOUS ALI (A/L)  \nNICHOLAS M. MAVODONES, JR (A/L)  \n \n \n \nORDER AMENDING ORDER 257 -20/21  \nFISCAL YEAR 2021- 2022 APPROPRIATION RESOLVE \n \n \nRESOLVED,  that the sum of $268,216,301 is hereby a ppropriated for Fiscal Year 202 2 \nfor General Municipal purposes; and  \n \nBE IT FURTHER RESOLVED,  that the sum of $125,154,700 $126,454,700 is hereby \nappropriated for Fiscal Year 2022 for School purposes; and  \n \nBE IT FURTHER RESOLVED,  that the Assessor of Taxes of the City of Portland be  \nand hereby is directed to assess a tax upon all real estate liable to be taxed \ntherein and to assess the owner of, or such other pe rsons as may be liable \nby law for, personal property liable to be taxed therein on 

In [22]:
df["Description"] = df["Description"].map(lambda Description: Description.lower())

In [23]:
words = [['nature', 'ecology', 'conversation', 'green', 'environment', 'suistainability'],
         ['installation', 'street', 'transportation', 'development', 'utility', 'street', 'infrastructure', 'construction', 'development', 'bridges', 'roads'],
         ['teaching', 'curriculum', 'school', 'education', 'learning', 'students'],
         ['housing', 'land', 'zoning', 'planning', 'regulations', 'permit', 'development', 'parking'],
         ['fire', 'emergency', 'health', 'security', 'safety', 'protection']]
category = ["Environmental","Infrastructure","Education","Zoning","Safety"]

d = {'category': category,'words': words}
keyword = pd.DataFrame(data=d)
keyword['words'] = keyword['words']
keyword['number_of_keywords'] = keyword['words'].apply(lambda row: len(row))

keyword.head()

,category,words,number_of_keywords
0,Environmental,"[nature, ecology, conversation, green, environ...",6
1,Infrastructure,"[installation, street, transportation, develop...",11
2,Education,"[teaching, curriculum, school, education, lear...",6
3,Zoning,"[housing, land, zoning, planning, regulations,...",8
4,Safety,"[fire, emergency, health, security, safety, pr...",6


In [24]:
tf = pd.DataFrame(df["category"].value_counts()).reset_index()
tf['category_count_letters'] = tf['category'].str.len()
tf

,category,count,category_count_letters
0,Education,102,9
1,Safety,75,6
2,Environmental,60,13
3,Infrastructure,57,14
4,Zoning,52,6


In [25]:
rf = pd.DataFrame(keyword["category"].value_counts()).reset_index()
rf['category_count_letters'] = rf['category'].str.len()
rf

,category,count,category_count_letters
0,Environmental,1,13
1,Infrastructure,1,14
2,Education,1,9
3,Zoning,1,6
4,Safety,1,6


In [26]:
keyword

,category,words,number_of_keywords
0,Environmental,"[nature, ecology, conversation, green, environ...",6
1,Infrastructure,"[installation, street, transportation, develop...",11
2,Education,"[teaching, curriculum, school, education, lear...",6
3,Zoning,"[housing, land, zoning, planning, regulations,...",8
4,Safety,"[fire, emergency, health, security, safety, pr...",6


In [27]:
keyword["category"].value_counts()

category
Environmental     1
Infrastructure    1
Education         1
Zoning            1
Safety            1
Name: count, dtype: int64

In [28]:
df["Description"][1]

' \norder 1 2-21/22 \npassage: 9 -0 on 7/19/2021      effective 7/29/2021 \n \nkate snyder (mayor)  \nbelinda s. ray (1)  \nspencer r. thibodeau (2)  \ntae y. chong (3)  \nandrew zarro (4)  city of portland  \nin the city council  mark dion (5)  \napril d. fournier(a/l)  \npious ali (a/l)  \nnicholas m. mavodones, jr (a/l)  \n \n \n \norder amending order 257 -20/21  \nfiscal year 2021- 2022 appropriation resolve \n \n \nresolved,  that the sum of $268,216,301 is hereby a ppropriated for fiscal year 202 2 \nfor general municipal purposes; and  \n \nbe it further resolved,  that the sum of $125,154,700 $126,454,700 is hereby \nappropriated for fiscal year 2022 for school purposes; and  \n \nbe it further resolved,  that the assessor of taxes of the city of portland be  \nand hereby is directed to assess a tax upon all real estate liable to be taxed \ntherein and to assess the owner of, or such other pe rsons as may be liable \nby law for, personal property liable to be taxed therein on 

In [29]:
# doc: document text string
# returns tokenized document
# strip_tags removes meta tags from the text
# simple preprocess converts a document into a list of lowercase tokens, ignoring tokens that are too short or too long 
# simple preprocess also removes numerical values as well as punktuation characters
def tokenize(doc):
    return simple_preprocess(strip_tags(doc), deacc=True, min_len=2, max_len=15)

In [30]:
# tokenize and tag documents combined title + description for Lbl2Vec training
df["Description_tok"] = df.apply(lambda row: TaggedDocument(tokenize(row['Description']), [str(row.name)]), axis=1)


In [32]:
#df['Description_tok'][10]

In [33]:
df.tail()

,link,category,Description,Description_tok
341,https://content.civicplus.com/api/assets/89003...,Safety,order 2 55-22/23 \npassage as an emergency : 7...,"([order, passage, as, an, emergency, dion, fou..."
342,https://content.civicplus.com/api/assets/a147d...,Education,order 260 -22/23 \nmotion to amend to set eff...,"([order, motion, to, amend, to, set, effective..."
343,https://content.civicplus.com/api/assets/a258d...,Education,orders \appointments \police chief sauschuck ...,"([orders, appointments, police, chief, sauschu..."
344,https://content.civicplus.com/api/assets/19ab9...,Education,order 263- 22/23 \npassage as an emergency : 7...,"([order, passage, as, an, emergency, dion, fou..."
345,https://content.civicplus.com/api/assets/175b1...,Environmental,order 173- 22/23 \npassage: 9 -0 on 4/24/2023 ...,"([order, passage, on, effective, kate, snyder,..."


In [34]:
df.reset_index(drop=True,inplace=True)
df['doc_key'] = df.index.astype(str)
df.head()

,link,category,Description,Description_tok,doc_key
0,https://content.civicplus.com/api/assets/bcff9...,Safety,order 1 1-21/22 \npassage: 9 -0 on 7/19/2021 ...,"([order, passage, on, effective, kate, snyder,...",0
1,https://content.civicplus.com/api/assets/f8a03...,Zoning,\norder 1 2-21/22 \npassage: 9 -0 on 7/19/202...,"([order, passage, on, effective, kate, snyder,...",1
2,https://content.civicplus.com/api/assets/cc288...,Safety,\n \n13432743.1 \norder 1 3-21/22 \npassage...,"([order, passage, on, effective, kate, snyder,...",2
3,https://content.civicplus.com/api/assets/0ea69...,Zoning,\n1 order 14-21/22 \npassage: 9-0 on 7/19/20...,"([order, passage, on, effective, kate, snyder,...",3
4,https://content.civicplus.com/api/assets/8ec46...,Education,order 15 -21/22 \nmotion to amend with amendme...,"([order, motion, to, amend, with, amendment, i...",4


In [35]:
df_train = df[:250]
df_train['data_set_type'] = 'train'
df_test = df[251:]
df_test['data_set_type'] = 'test'

In [36]:
print("train data shape",df_train.shape)
print("test data shape",df_test.shape)


# concat train and test data
ag_full_corpus = pd.concat([df_train,df_test]).reset_index(drop=True)

ag_full_corpus.head()

train data shape (250, 6)
test data shape (95, 6)


,link,category,Description,Description_tok,doc_key,data_set_type
0,https://content.civicplus.com/api/assets/bcff9...,Safety,order 1 1-21/22 \npassage: 9 -0 on 7/19/2021 ...,"([order, passage, on, effective, kate, snyder,...",0,train
1,https://content.civicplus.com/api/assets/f8a03...,Zoning,\norder 1 2-21/22 \npassage: 9 -0 on 7/19/202...,"([order, passage, on, effective, kate, snyder,...",1,train
2,https://content.civicplus.com/api/assets/cc288...,Safety,\n \n13432743.1 \norder 1 3-21/22 \npassage...,"([order, passage, on, effective, kate, snyder,...",2,train
3,https://content.civicplus.com/api/assets/0ea69...,Zoning,\n1 order 14-21/22 \npassage: 9-0 on 7/19/20...,"([order, passage, on, effective, kate, snyder,...",3,train
4,https://content.civicplus.com/api/assets/8ec46...,Education,order 15 -21/22 \nmotion to amend with amendme...,"([order, motion, to, amend, with, amendment, i...",4,train


In [37]:
ag_full_corpus.dtypes

link               object
category           object
Description        object
Description_tok    object
doc_key            object
data_set_type      object
dtype: object

In [38]:
# init model with parameters
#lbl2vec_model = Lbl2Vec(keywords_list=list(labels['keywords']), 
#                        tagged_documents=ag_full_corpus['tagged_docs'][ag_full_corpus['data_set_type']=='train'],
#                        label_names=list(labels['class_name']), similarity_threshold=0.30, min_num_docs=100, epochs=10)
#

In [39]:
keyword.head()

,category,words,number_of_keywords
0,Environmental,"[nature, ecology, conversation, green, environ...",6
1,Infrastructure,"[installation, street, transportation, develop...",11
2,Education,"[teaching, curriculum, school, education, lear...",6
3,Zoning,"[housing, land, zoning, planning, regulations,...",8
4,Safety,"[fire, emergency, health, security, safety, pr...",6


# LABEL 2 VECTOR

In [40]:
# init model with parameters
lbl2vec_model = Lbl2Vec(keywords_list=list(keyword['words']),
                        tagged_documents=ag_full_corpus['Description_tok'][ag_full_corpus['data_set_type']=='train'],
                        label_names=list(keyword['category']),min_count = 2,similarity_threshold=0.20, min_num_docs=32, epochs=10)


In [41]:
lbl2vec_model.fit()

2023-12-14 03:47:43,658 - Lbl2Vec - INFO - Train document and word embeddings
2023-12-14 03:48:59,661 - Lbl2Vec - INFO - Train label embeddings
2023-12-14 03:48:59,661 - Lbl2Vec - WARNING - The following keywords from the 'keywords_list' are unknown to the Doc2Vec model and therefore not used to train the model: ecology suistainability
2023-12-14 03:48:59,735 - Lbl2Vec - WARNING - The following keywords from the 'keywords_list' are unknown to the Doc2Vec model and therefore not used to train the model: curriculum


### Predict topics of documents used to train Lbl2Vec


Compute similarity scores of learned document vectors from documents that were used to train the model to each of the learned label vectors. The similarity scores consist of cosine similarities and therefore have a value range of [-1,1].

In [43]:
# predict similarity scores
model_docs_lbl_similarities = lbl2vec_model.predict_model_docs()

2023-12-14 03:51:26,843 - Lbl2Vec - INFO - Get document embeddings from model
2023-12-14 03:51:26,850 - Lbl2Vec - INFO - Calculate document<->label similarities


In [44]:
model_docs_lbl_similarities.head()

,doc_key,most_similar_label,highest_similarity_score,Environmental,Infrastructure,Education,Zoning,Safety
0,0,Safety,0.774035,0.742092,0.728910,0.765886,0.741531,0.774035
1,1,Environmental,0.564886,0.564886,0.503945,0.515262,0.537485,0.505996
2,2,Environmental,0.692392,0.692392,0.649376,0.674425,0.653328,0.688099
3,3,Infrastructure,0.626605,0.543646,0.626605,0.617145,0.625556,0.599429
4,4,Education,0.465770,0.432288,0.431041,0.465770,0.451420,0.445979


### Evaluate prediction of documents used to train Lbl2Vec

In [45]:
# merge DataFrames to compare the predicted and true topic labels
evaluation_train = model_docs_lbl_similarities.merge(ag_full_corpus[ag_full_corpus['data_set_type']=='train'], left_on='doc_key', right_on='doc_key')


In [46]:
evaluation_train.head(2)

,doc_key,most_similar_label,highest_similarity_score,Environmental,Infrastructure,Education,Zoning,Safety,link,category,Description,Description_tok,data_set_type
0,0,Safety,0.774035,0.742092,0.728910,0.765886,0.741531,0.774035,https://content.civicplus.com/api/assets/bcff9...,Safety,order 1 1-21/22 \npassage: 9 -0 on 7/19/2021 ...,"([order, passage, on, effective, kate, snyder,...",train
1,1,Environmental,0.564886,0.564886,0.503945,0.515262,0.537485,0.505996,https://content.civicplus.com/api/assets/f8a03...,Zoning,\norder 1 2-21/22 \npassage: 9 -0 on 7/19/202...,"([order, passage, on, effective, kate, snyder,...",train


In [47]:
y_true_train = evaluation_train['category']
y_pred_train = evaluation_train['most_similar_label']
print('F1 score:',f1_score(y_true_train, y_pred_train, average='micro'))

F1 score: 0.544


In [48]:
# predict similarity scores of new test documents (they were not used during Lbl2Vec training)
new_docs_lbl_similarities = lbl2vec_model.predict_new_docs(tagged_docs=ag_full_corpus['Description_tok'][ag_full_corpus['data_set_type']=='test'])


2023-12-14 03:51:30,311 - Lbl2Vec - INFO - Calculate document embeddings
2023-12-14 03:51:33,574 - Lbl2Vec - INFO - Calculate document<->label similarities


In [49]:
new_docs_lbl_similarities.head()

,doc_key,most_similar_label,highest_similarity_score,Environmental,Infrastructure,Education,Zoning,Safety
0,251,Safety,0.602468,0.576034,0.577311,0.597208,0.579584,0.602468
1,252,Safety,0.722294,0.696671,0.701737,0.712892,0.697790,0.722294
2,253,Zoning,0.234598,0.227706,0.222269,0.210095,0.234598,0.207891
3,254,Infrastructure,0.241501,0.241436,0.241501,0.220715,0.228321,0.200641
4,255,Infrastructure,0.298345,0.281899,0.298345,0.256753,0.297620,0.221819


### Evaluate prediction of new documents

In [50]:
# merge DataFrames to compare the predicted and true topic labels
evaluation_test = new_docs_lbl_similarities.merge(ag_full_corpus[ag_full_corpus['data_set_type']=='test'], left_on='doc_key', right_on='doc_key')


In [51]:
y_true_test = evaluation_test['category']
y_pred_test = evaluation_test['most_similar_label']
print('F1 score:',f1_score(y_true_test, y_pred_test, average='micro'))

F1 score: 0.5473684210526316


##### Let's Check Accuracy 

In [52]:
print(evaluation_test.shape)
print(evaluation_train.shape)

(95, 13)
(250, 13)


In [53]:
final_frame = pd.concat([evaluation_train,evaluation_test])

In [54]:
final_frame.head()

,doc_key,most_similar_label,highest_similarity_score,Environmental,Infrastructure,Education,Zoning,Safety,link,category,Description,Description_tok,data_set_type
0,0,Safety,0.774035,0.742092,0.728910,0.765886,0.741531,0.774035,https://content.civicplus.com/api/assets/bcff9...,Safety,order 1 1-21/22 \npassage: 9 -0 on 7/19/2021 ...,"([order, passage, on, effective, kate, snyder,...",train
1,1,Environmental,0.564886,0.564886,0.503945,0.515262,0.537485,0.505996,https://content.civicplus.com/api/assets/f8a03...,Zoning,\norder 1 2-21/22 \npassage: 9 -0 on 7/19/202...,"([order, passage, on, effective, kate, snyder,...",train
2,2,Environmental,0.692392,0.692392,0.649376,0.674425,0.653328,0.688099,https://content.civicplus.com/api/assets/cc288...,Safety,\n \n13432743.1 \norder 1 3-21/22 \npassage...,"([order, passage, on, effective, kate, snyder,...",train
3,3,Infrastructure,0.626605,0.543646,0.626605,0.617145,0.625556,0.599429,https://content.civicplus.com/api/assets/0ea69...,Zoning,\n1 order 14-21/22 \npassage: 9-0 on 7/19/20...,"([order, passage, on, effective, kate, snyder,...",train
4,4,Education,0.465770,0.432288,0.431041,0.465770,0.451420,0.445979,https://content.civicplus.com/api/assets/8ec46...,Education,order 15 -21/22 \nmotion to amend with amendme...,"([order, motion, to, amend, with, amendment, i...",train


In [55]:
final_frame[["link","most_similar_label"]].to_csv("data_election_bridge.csv")

In [56]:
new_frame = final_frame[["category","most_similar_label"]]


In [57]:
new_frame.loc[new_frame.category == new_frame.most_similar_label, "score"] = 1
new_frame.loc[new_frame.category != new_frame.most_similar_label, "score"] = 0

In [58]:
new_frame.head()

,category,most_similar_label,score
0,Safety,Safety,1.0
1,Zoning,Environmental,0.0
2,Safety,Environmental,0.0
3,Zoning,Infrastructure,0.0
4,Education,Education,1.0


In [59]:
### Accuracy is simple 

new_frame["score"].sum()/len(new_frame)

0.5449275362318841

# LABEL 2 Transformer VECTOR

In [60]:
X_train, X_test, y_train, y_test = train_test_split(ag_full_corpus['Description'],ag_full_corpus['category'], test_size=0.10, random_state=42)

In [61]:
#init model with parameters
model = Lbl2TransformerVec(keywords_list=list(keyword['words']), documents=X_train)

# train model
model.fit()

2023-12-14 03:52:04,025 - Lbl2TransformerVec - INFO - Compute keyword embeddings
2023-12-14 03:52:05,410 - Lbl2TransformerVec - INFO - Compute document embeddings
2023-12-14 04:07:45,958 - Lbl2TransformerVec - INFO - Train label embeddings


In [62]:
# predict similarity scores
model_docs_lbl_similarities = model.predict_model_docs()



2023-12-14 04:11:12,786 - Lbl2TransformerVec - INFO - Get document embeddings from model
2023-12-14 04:11:12,793 - Lbl2TransformerVec - INFO - Calculate document<->label similarities


In [63]:
keyword

,category,words,number_of_keywords
0,Environmental,"[nature, ecology, conversation, green, environ...",6
1,Infrastructure,"[installation, street, transportation, develop...",11
2,Education,"[teaching, curriculum, school, education, lear...",6
3,Zoning,"[housing, land, zoning, planning, regulations,...",8
4,Safety,"[fire, emergency, health, security, safety, pr...",6


In [64]:
model_docs_lbl_similarities.sample(1)

,doc_key,most_similar_label,highest_similarity_score,label_0,label_1,label_2,label_3,label_4
201,201,label_4,0.759605,0.702538,0.732785,0.756705,0.317061,0.759605


In [65]:
model_docs_lbl_similarities['prediction'] = model_docs_lbl_similarities['most_similar_label'].map({
    'label_0': 'Environmental', 'label_1': 'Infrastructure','label_2':'Education','label_3':'Zoning','label_4':'Safety'})
model_docs_lbl_similarities.head()

,doc_key,most_similar_label,highest_similarity_score,label_0,label_1,label_2,label_3,label_4,prediction
0,0,label_0,0.968625,0.968625,0.770401,0.707191,0.308134,0.831536,Environmental
1,1,label_1,0.529829,0.509282,0.529829,0.488037,0.435918,0.509213,Infrastructure
2,2,label_0,0.414925,0.414925,0.338329,0.290241,0.312018,0.413073,Environmental
3,3,label_4,0.784255,0.773353,0.765571,0.712455,0.292661,0.784255,Safety
4,4,label_0,0.778304,0.778304,0.740584,0.661643,0.245599,0.725083,Environmental


In [66]:
model_docs_lbl_similarities["prediction"].value_counts()

prediction
Environmental     130
Infrastructure     68
Zoning             41
Safety             40
Education          31
Name: count, dtype: int64

In [67]:
# evaluation_train = model_docs_lbl_similarities.merge(df, left_on='doc_key', right_on='doc_key')
y_true_train = y_train
y_pred_train = model_docs_lbl_similarities['prediction']
print('F1 score:',f1_score(y_true_train, y_pred_train, average='micro'))

F1 score: 0.23548387096774193


In [68]:
d = {'TRUE': y_train,'PRED': y_pred_train}
ff = pd.DataFrame(data=d)

ff.loc[ff.TRUE == ff.PRED, "score"] = 1
ff.loc[ff.TRUE != ff.PRED, "score"] = 0

### Accuracy is simple 

new_frame["score"].sum()/len(new_frame)

0.5449275362318841

### TRUE LABLE DATA 40 POLICY

In [33]:
df = pd.read_excel('TRUE_LABEL_BY_SPONSOR.xlsx')

In [34]:
df.head()

,link,category,title
0,https://content.civicplus.com/api/assets/b2e87...,Zoning and infrastructure,AMENDMENT TO PORTLAND CITY CODE CHAPTER 8\nRe:...
1,https://content.civicplus.com/api/assets/ea087...,Environmental,"ORDER APPROVING THE APPLICATION FOR, ACCEPTANC..."
2,https://content.civicplus.com/api/assets/299d3...,Infrastructure,ORDER APPROVING MODIFICATION TO TWO-PARTY AGRE...
3,https://content.civicplus.com/api/assets/e1184...,Infrastructure,ORDER ACCEPTING AND ADOPTING THE 2020 HOUSING ...
4,https://content.civicplus.com/api/assets/1768f...,Infrastructure,"ORDER APPROPRIATING $36,000 FROM THE HOUSING T..."


In [35]:
df["category"].unique()

array(['Zoning and infrastructure', 'Environmental', 'Infrastructure',
       'Safety', 'Zoning', 'Education', 'Education/Infrastructure'],
      dtype=object)

In [36]:
df["category"].value_counts()

category
Safety                       10
Infrastructure                9
Education                     8
Environmental                 5
Zoning                        4
Zoning and infrastructure     1
Education/Infrastructure      1
Name: count, dtype: int64

In [37]:
df = df[(df["category"].isin(['Education', 'Infrastructure', 'Zoning', 'Safety', 'Environmental']))]

In [38]:
df["category"].value_counts()

category
Safety            10
Infrastructure     9
Education          8
Environmental      5
Zoning             4
Name: count, dtype: int64

In [39]:
df.shape

(36, 3)

In [40]:
texts = []
for i in tqdm(df['link']):
    response = requests.get(i)
    pdf_io_bytes = io.BytesIO(response.content)
    text_list = []
    pdf = PyPDF2.PdfReader(pdf_io_bytes)

    num_pages = len(pdf.pages)

    for page in range(num_pages):
        page_text = pdf.pages[page].extract_text()
        text_list.append(page_text)
    text = "\n".join(text_list)
    texts.append(text)

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:32<00:00,  1.11it/s]


In [41]:
len(texts)

36

In [42]:
#texts

In [43]:
df["Description"] = texts

In [44]:
df.head()

,link,category,title,Description
1,https://content.civicplus.com/api/assets/ea087...,Environmental,"ORDER APPROVING THE APPLICATION FOR, ACCEPTANC...",Order 84 -19/20 \nPassage: 8 -0 (Cook absent)...
2,https://content.civicplus.com/api/assets/299d3...,Infrastructure,ORDER APPROVING MODIFICATION TO TWO-PARTY AGRE...,Order 83 -19/20 \nPassage: 8 -0 (Cook absent) ...
3,https://content.civicplus.com/api/assets/e1184...,Infrastructure,ORDER ACCEPTING AND ADOPTING THE 2020 HOUSING ...,\nOrder 146 -19/20 \nPassage: 9 -0 on 3/ 2/20...
4,https://content.civicplus.com/api/assets/1768f...,Infrastructure,"ORDER APPROPRIATING $36,000 FROM THE HOUSING T...",Order 54 -19/20 \nPassage: 8 -0 (Ali absent) o...
5,https://content.civicplus.com/api/assets/af3b4...,Safety,AMENDMENT TO PORTLAND CITY CODE CHAPTER 10 RE:...,Order 59 -19/20 \nMotion to postpone to Novem...


In [45]:
df.tail()

,link,category,title,Description
33,https://content.civicplus.com/api/assets/e1184...,Infrastructure,ORDER APPROVING AN OPTION TO LEASE AGREEMENT W...,\nOrder 146 -19/20 \nPassage: 9 -0 on 3/ 2/20...
34,https://content.civicplus.com/api/assets/e1184...,Infrastructure,ORDER ACCEPTING AND ADOPTING THE 2020 HOUSING ...,\nOrder 146 -19/20 \nPassage: 9 -0 on 3/ 2/20...
35,https://content.civicplus.com/api/assets/18278...,Safety,ORDER ACCEPTING A HOUSING AND URBAN DEVELOPMEN...,Order 103 -19/20 \nMotion to waive the second ...
36,https://content.civicplus.com/api/assets/c73c8...,Infrastructure,"ORDER APPROPRIATING $193,266 IN HOME INVESTMEN...",Order 34 -19/20 \nPassage: 9 -0 on 9/4/2019 ...
37,https://content.civicplus.com/api/assets/8c493...,Zoning,ORDER APPROVING THE SECOND AMENDMENT TO CONDIT...,Order 13 -19/20 \nPassag e: 7-0 (Batson and Co...


In [46]:
df["Description"] = df["Description"].map(lambda Description: Description.lower())

In [47]:
words = [['nature', 'ecology', 'conversation', 'green', 'environment', 'suistainability'],['installation', 'street', 'transportation', 'development', 'utility', 'street', 'infrastructure', 'construction', 'development', 'bridges', 'roads'],['teaching', 'curriculum', 'school', 'education', 'learning', 'students'],['housing', 'land', 'zoning', 'planning', 'regulations', 'permit', 'development', 'parking'],['fire', 'emergency', 'health', 'security', 'safety', 'protection']]
category = ["Environmental","Infrastructure","Education","Zoning","Safety"]

d = {'category': category,'words': words}
keyword = pd.DataFrame(data=d)
keyword['words'] = keyword['words']
keyword['number_of_keywords'] = keyword['words'].apply(lambda row: len(row))

keyword.head()

,category,words,number_of_keywords
0,Environmental,"[nature, ecology, conversation, green, environ...",6
1,Infrastructure,"[installation, street, transportation, develop...",11
2,Education,"[teaching, curriculum, school, education, lear...",6
3,Zoning,"[housing, land, zoning, planning, regulations,...",8
4,Safety,"[fire, emergency, health, security, safety, pr...",6


In [48]:
tf = pd.DataFrame(df["category"].value_counts()).reset_index()
tf['category_count_letters'] = tf['category'].str.len()
tf

,category,count,category_count_letters
0,Safety,10,6
1,Infrastructure,9,14
2,Education,8,9
3,Environmental,5,13
4,Zoning,4,6


In [49]:
rf = pd.DataFrame(keyword["category"].value_counts()).reset_index()
rf['category_count_letters'] = rf['category'].str.len()
rf

,category,count,category_count_letters
0,Environmental,1,13
1,Infrastructure,1,14
2,Education,1,9
3,Zoning,1,6
4,Safety,1,6


In [50]:
keyword

,category,words,number_of_keywords
0,Environmental,"[nature, ecology, conversation, green, environ...",6
1,Infrastructure,"[installation, street, transportation, develop...",11
2,Education,"[teaching, curriculum, school, education, lear...",6
3,Zoning,"[housing, land, zoning, planning, regulations,...",8
4,Safety,"[fire, emergency, health, security, safety, pr...",6


In [51]:
keyword["category"].value_counts()

category
Environmental     1
Infrastructure    1
Education         1
Zoning            1
Safety            1
Name: count, dtype: int64

In [52]:
df["Description"][1]

' order 84 -19/20 \npassage: 8 -0 (cook absent) on 11/18/2019    effective 11/28/2019  \n \nethan k. strimling (mayor)  \nbelinda s. ray (1)  \nspencer r. thibodeau (2)  \nbrian e. batson (3)  \njustin costa (4)  city of portland  \nin the city council  kimberly cook (5)  \njill c. duson (a/l)  \npious ali (a/l)  \nnicholas m. mavodones, jr (a/l)  \n \n \n \norder approving th e application for, accept ance  and \nappropriation of an up to $250,000 brownfield  assessment  fund \ngrant  \n \n \n \nordered, that the application for up to $250,000 brownfields assessment grant from the \nunited states environmental protection agency is hereby approved; and  \n \nbe it further ordered, that a brownfields assessment grant in the amount of up to \n$250,000 from the u.s. environmental protection agency, if awarded, is hereby \naccepted for the purposes of funding assessments of contaminated sites; and \n \nbe it furth er ordered, that, if awarded and accepted, the brownfields assessment \ngran

In [53]:
# doc: document text string
# returns tokenized document
# strip_tags removes meta tags from the text
# simple preprocess converts a document into a list of lowercase tokens, ignoring tokens that are too short or too long 
# simple preprocess also removes numerical values as well as punktuation characters
def tokenize(doc):
    return simple_preprocess(strip_tags(doc), deacc=True, min_len=2, max_len=15)

In [54]:
# tokenize and tag documents combined title + description for Lbl2Vec training
df["Description_tok"] = df.apply(lambda row: TaggedDocument(tokenize(row['Description']), [str(row.name)]), axis=1)


In [55]:
df['Description_tok'][10]

TaggedDocument(words=['order', 'passage', 'on', 'effective', 'kate', 'snyder', 'mayor', 'belinda', 'ray', 'spencer', 'thibodeau', 'tae', 'chong', 'justin', 'costa', 'city', 'of', 'portland', 'in', 'the', 'city', 'council', 'kimberly', 'cook', 'jill', 'duson', 'pious', 'ali', 'nicholas', 'mavodones', 'jr', 'order', 'ccept', 'ing', 'and', 'appropriati', 'ng', 'grant', 'from', 'the', 'sam', 'cohen', 'foundation', 'for', 'the', 'english', 'speakers', 'of', 'other', 'languages', 'collaborative', 'project', 'ordered', 'that', 'sam', 'cohen', 'foundation', 'grant', 'in', 'the', 'amount', 'of', 'is', 'hereby', 'accepted', 'for', 'the', 'engli', 'sh', 'speakers', 'of', 'other', 'language', 'collaborative', 'pr', 'oject', 'and', 'be', 'it', 'further', 'ordered', 'that', 'the', 'is', 'hereby', 'appropriated', 'for', 'use', 'by', 'the', 'engli', 'sh', 'speakers', 'of', 'other', 'languages', 'collaborative', 'roject', 'and', 'be', 'it', 'further', 'ordered', 'the', 'city', 'manager', 'or', 'his', '

In [56]:
df.tail()

,link,category,title,Description,Description_tok
33,https://content.civicplus.com/api/assets/e1184...,Infrastructure,ORDER APPROVING AN OPTION TO LEASE AGREEMENT W...,\norder 146 -19/20 \npassage: 9 -0 on 3/ 2/20...,"([order, passage, on, effective, kate, snyder,..."
34,https://content.civicplus.com/api/assets/e1184...,Infrastructure,ORDER ACCEPTING AND ADOPTING THE 2020 HOUSING ...,\norder 146 -19/20 \npassage: 9 -0 on 3/ 2/20...,"([order, passage, on, effective, kate, snyder,..."
35,https://content.civicplus.com/api/assets/18278...,Safety,ORDER ACCEPTING A HOUSING AND URBAN DEVELOPMEN...,order 103 -19/20 \nmotion to waive the second ...,"([order, motion, to, waive, the, second, readi..."
36,https://content.civicplus.com/api/assets/c73c8...,Infrastructure,"ORDER APPROPRIATING $193,266 IN HOME INVESTMEN...",order 34 -19/20 \npassage: 9 -0 on 9/4/2019 ...,"([order, passage, on, effective, ethan, striml..."
37,https://content.civicplus.com/api/assets/8c493...,Zoning,ORDER APPROVING THE SECOND AMENDMENT TO CONDIT...,order 13 -19/20 \npassag e: 7-0 (batson and co...,"([order, passag, batson, and, costa, absent, o..."


In [57]:
df.reset_index(drop=True,inplace=True)
df['doc_key'] = df.index.astype(str)
df.head()

,link,category,title,Description,Description_tok,doc_key
0,https://content.civicplus.com/api/assets/ea087...,Environmental,"ORDER APPROVING THE APPLICATION FOR, ACCEPTANC...",order 84 -19/20 \npassage: 8 -0 (cook absent)...,"([order, passage, cook, absent, on, effective,...",0
1,https://content.civicplus.com/api/assets/299d3...,Infrastructure,ORDER APPROVING MODIFICATION TO TWO-PARTY AGRE...,order 83 -19/20 \npassage: 8 -0 (cook absent) ...,"([order, passage, cook, absent, on, effective,...",1
2,https://content.civicplus.com/api/assets/e1184...,Infrastructure,ORDER ACCEPTING AND ADOPTING THE 2020 HOUSING ...,\norder 146 -19/20 \npassage: 9 -0 on 3/ 2/20...,"([order, passage, on, effective, kate, snyder,...",2
3,https://content.civicplus.com/api/assets/1768f...,Infrastructure,"ORDER APPROPRIATING $36,000 FROM THE HOUSING T...",order 54 -19/20 \npassage: 8 -0 (ali absent) o...,"([order, passage, ali, absent, on, effective, ...",3
4,https://content.civicplus.com/api/assets/af3b4...,Safety,AMENDMENT TO PORTLAND CITY CODE CHAPTER 10 RE:...,order 59 -19/20 \nmotion to postpone to novem...,"([order, motion, to, postpone, to, november, a...",4


In [73]:
df_train = df[:250]
df_train['data_set_type'] = 'train'

In [74]:
print("train data shape",df_train.shape)
print("test data shape",df_test.shape)


# concat train and test data
ag_full_corpus = pd.concat([df_train,df_test]).reset_index(drop=True)

ag_full_corpus.head()

train data shape (36, 7)
test data shape (0, 7)


,link,category,title,Description,Description_tok,doc_key,data_set_type
0,https://content.civicplus.com/api/assets/ea087...,Environmental,"ORDER APPROVING THE APPLICATION FOR, ACCEPTANC...",order 84 -19/20 \npassage: 8 -0 (cook absent)...,"([order, passage, cook, absent, on, effective,...",0,train
1,https://content.civicplus.com/api/assets/299d3...,Infrastructure,ORDER APPROVING MODIFICATION TO TWO-PARTY AGRE...,order 83 -19/20 \npassage: 8 -0 (cook absent) ...,"([order, passage, cook, absent, on, effective,...",1,train
2,https://content.civicplus.com/api/assets/e1184...,Infrastructure,ORDER ACCEPTING AND ADOPTING THE 2020 HOUSING ...,\norder 146 -19/20 \npassage: 9 -0 on 3/ 2/20...,"([order, passage, on, effective, kate, snyder,...",2,train
3,https://content.civicplus.com/api/assets/1768f...,Infrastructure,"ORDER APPROPRIATING $36,000 FROM THE HOUSING T...",order 54 -19/20 \npassage: 8 -0 (ali absent) o...,"([order, passage, ali, absent, on, effective, ...",3,train
4,https://content.civicplus.com/api/assets/af3b4...,Safety,AMENDMENT TO PORTLAND CITY CODE CHAPTER 10 RE:...,order 59 -19/20 \nmotion to postpone to novem...,"([order, motion, to, postpone, to, november, a...",4,train


In [75]:
ag_full_corpus.dtypes

link               object
category           object
title              object
Description        object
Description_tok    object
doc_key            object
data_set_type      object
dtype: object

In [76]:
# init model with parameters
#lbl2vec_model = Lbl2Vec(keywords_list=list(labels['keywords']), 
#                        tagged_documents=ag_full_corpus['tagged_docs'][ag_full_corpus['data_set_type']=='train'],
#                        label_names=list(labels['class_name']), similarity_threshold=0.30, min_num_docs=100, epochs=10)
#

In [77]:
keyword.head()

,category,words,number_of_keywords
0,Environmental,"[nature, ecology, conversation, green, environ...",6
1,Infrastructure,"[installation, street, transportation, develop...",11
2,Education,"[teaching, curriculum, school, education, lear...",6
3,Zoning,"[housing, land, zoning, planning, regulations,...",8
4,Safety,"[fire, emergency, health, security, safety, pr...",6


# LABEL 2 VECTOR

In [78]:
# init model with parameters
lbl2vec_model = Lbl2Vec(keywords_list=list(keyword['words']),
                        tagged_documents=ag_full_corpus['Description_tok'][ag_full_corpus['data_set_type']=='train'],
                        label_names=list(keyword['category']),min_count = 2,similarity_threshold=0.20, min_num_docs=32, epochs=10)


In [79]:
lbl2vec_model.fit()

2023-12-14 03:21:08,870 - Lbl2Vec - INFO - Train document and word embeddings
2023-12-14 03:21:08,870 - Lbl2Vec - INFO - Train document and word embeddings
2023-12-14 03:21:13,854 - Lbl2Vec - INFO - Train label embeddings
2023-12-14 03:21:13,854 - Lbl2Vec - INFO - Train label embeddings
2023-12-14 03:21:13,855 - Lbl2Vec - WARNING - The following keywords from the 'keywords_list' are unknown to the Doc2Vec model and therefore not used to train the model: ecology conversation environment suistainability
2023-12-14 03:21:13,855 - Lbl2Vec - WARNING - The following keywords from the 'keywords_list' are unknown to the Doc2Vec model and therefore not used to train the model: ecology conversation environment suistainability
2023-12-14 03:21:13,860 - Lbl2Vec - WARNING - The following keywords from the 'keywords_list' are unknown to the Doc2Vec model and therefore not used to train the model: bridges
2023-12-14 03:21:13,860 - Lbl2Vec - WARNING - The following keywords from the 'keywords_list' ar

### Predict topics of documents used to train Lbl2Vec


Compute similarity scores of learned document vectors from documents that were used to train the model to each of the learned label vectors. The similarity scores consist of cosine similarities and therefore have a value range of [-1,1].

In [80]:
# predict similarity scores
model_docs_lbl_similarities = lbl2vec_model.predict_model_docs()

2023-12-14 03:21:13,882 - Lbl2Vec - INFO - Get document embeddings from model
2023-12-14 03:21:13,882 - Lbl2Vec - INFO - Get document embeddings from model
2023-12-14 03:21:13,891 - Lbl2Vec - INFO - Calculate document<->label similarities
2023-12-14 03:21:13,891 - Lbl2Vec - INFO - Calculate document<->label similarities


In [81]:
model_docs_lbl_similarities.head()

,doc_key,most_similar_label,highest_similarity_score,Environmental,Infrastructure,Education,Zoning,Safety
0,1,Education,0.895171,0.881649,0.881507,0.895171,0.881507,0.874936
1,2,Safety,0.736441,0.690863,0.687935,0.707438,0.687935,0.736441
2,3,Education,0.431575,0.422219,0.415309,0.431575,0.415309,0.231189
3,4,Education,0.953316,0.937277,0.939575,0.953316,0.939575,0.940014
4,5,Environmental,0.380884,0.380884,0.379719,0.307597,0.379719,0.369057


### Evaluate prediction of documents used to train Lbl2Vec

In [82]:
# merge DataFrames to compare the predicted and true topic labels
evaluation_train = model_docs_lbl_similarities.merge(ag_full_corpus[ag_full_corpus['data_set_type']=='train'], left_on='doc_key', right_on='doc_key')


In [83]:
evaluation_train.head(2)

,doc_key,most_similar_label,highest_similarity_score,Environmental,Infrastructure,Education,Zoning,Safety,link,category,title,Description,Description_tok,data_set_type
0,1,Education,0.895171,0.881649,0.881507,0.895171,0.881507,0.874936,https://content.civicplus.com/api/assets/299d3...,Infrastructure,ORDER APPROVING MODIFICATION TO TWO-PARTY AGRE...,order 83 -19/20 \npassage: 8 -0 (cook absent) ...,"([order, passage, cook, absent, on, effective,...",train
1,2,Safety,0.736441,0.690863,0.687935,0.707438,0.687935,0.736441,https://content.civicplus.com/api/assets/e1184...,Infrastructure,ORDER ACCEPTING AND ADOPTING THE 2020 HOUSING ...,\norder 146 -19/20 \npassage: 9 -0 on 3/ 2/20...,"([order, passage, on, effective, kate, snyder,...",train


In [84]:
y_true_train = evaluation_train['category']
y_pred_train = evaluation_train['most_similar_label']
print('F1 score:',f1_score(y_true_train, y_pred_train, average='micro'))

F1 score: 0.35294117647058826


##### Let's Check Accuracy 

In [85]:
final_frame = evaluation_train.copy()
final_frame.head()

,doc_key,most_similar_label,highest_similarity_score,Environmental,Infrastructure,Education,Zoning,Safety,link,category,title,Description,Description_tok,data_set_type
0,1,Education,0.895171,0.881649,0.881507,0.895171,0.881507,0.874936,https://content.civicplus.com/api/assets/299d3...,Infrastructure,ORDER APPROVING MODIFICATION TO TWO-PARTY AGRE...,order 83 -19/20 \npassage: 8 -0 (cook absent) ...,"([order, passage, cook, absent, on, effective,...",train
1,2,Safety,0.736441,0.690863,0.687935,0.707438,0.687935,0.736441,https://content.civicplus.com/api/assets/e1184...,Infrastructure,ORDER ACCEPTING AND ADOPTING THE 2020 HOUSING ...,\norder 146 -19/20 \npassage: 9 -0 on 3/ 2/20...,"([order, passage, on, effective, kate, snyder,...",train
2,3,Education,0.431575,0.422219,0.415309,0.431575,0.415309,0.231189,https://content.civicplus.com/api/assets/1768f...,Infrastructure,"ORDER APPROPRIATING $36,000 FROM THE HOUSING T...",order 54 -19/20 \npassage: 8 -0 (ali absent) o...,"([order, passage, ali, absent, on, effective, ...",train
3,4,Education,0.953316,0.937277,0.939575,0.953316,0.939575,0.940014,https://content.civicplus.com/api/assets/af3b4...,Safety,AMENDMENT TO PORTLAND CITY CODE CHAPTER 10 RE:...,order 59 -19/20 \nmotion to postpone to novem...,"([order, motion, to, postpone, to, november, a...",train
4,5,Environmental,0.380884,0.380884,0.379719,0.307597,0.379719,0.369057,https://content.civicplus.com/api/assets/efb4d...,Environmental,AMENDMENT TO PORTLAND CITY CODE CHAPTER 12. GA...,order 61 -19/20 \nmotion to amend by adding a...,"([order, motion, to, amend, by, adding, new, b...",train


In [86]:
new_frame = final_frame[["category","most_similar_label"]]


In [87]:
new_frame.loc[new_frame.category == new_frame.most_similar_label, "score"] = 1
new_frame.loc[new_frame.category != new_frame.most_similar_label, "score"] = 0

In [88]:
new_frame.head()

,category,most_similar_label,score
0,Infrastructure,Education,0.0
1,Infrastructure,Safety,0.0
2,Infrastructure,Education,0.0
3,Safety,Education,0.0
4,Environmental,Environmental,1.0


In [89]:
### Accuracy is simple 

new_frame["score"].sum()/len(new_frame)

0.35294117647058826

# LABEL 2 Transformer VECTOR

In [98]:
X_train, X_test, y_train, y_test = train_test_split(ag_full_corpus['Description'],ag_full_corpus['category'], test_size=0.001, random_state=42)

In [99]:
#init model with parameters
model = Lbl2TransformerVec(keywords_list=list(keyword['words']), documents=X_train)

# train model
model.fit()

2023-12-14 03:25:26,452 - Lbl2TransformerVec - INFO - Compute keyword embeddings
2023-12-14 03:25:26,452 - Lbl2TransformerVec - INFO - Compute keyword embeddings
2023-12-14 03:25:29,045 - Lbl2TransformerVec - INFO - Compute document embeddings
2023-12-14 03:25:29,045 - Lbl2TransformerVec - INFO - Compute document embeddings
2023-12-14 03:27:42,743 - Lbl2TransformerVec - INFO - Train label embeddings
2023-12-14 03:27:42,743 - Lbl2TransformerVec - INFO - Train label embeddings


In [100]:
# predict similarity scores
model_docs_lbl_similarities = model.predict_model_docs()



2023-12-14 03:27:42,802 - Lbl2TransformerVec - INFO - Get document embeddings from model
2023-12-14 03:27:42,802 - Lbl2TransformerVec - INFO - Get document embeddings from model
2023-12-14 03:27:42,805 - Lbl2TransformerVec - INFO - Calculate document<->label similarities
2023-12-14 03:27:42,805 - Lbl2TransformerVec - INFO - Calculate document<->label similarities


In [101]:
keyword

,category,words,number_of_keywords
0,Environmental,"[nature, ecology, conversation, green, environ...",6
1,Infrastructure,"[installation, street, transportation, develop...",11
2,Education,"[teaching, curriculum, school, education, lear...",6
3,Zoning,"[housing, land, zoning, planning, regulations,...",8
4,Safety,"[fire, emergency, health, security, safety, pr...",6


In [102]:
model_docs_lbl_similarities.sample(1)

,doc_key,most_similar_label,highest_similarity_score,label_0,label_1,label_2,label_3,label_4
22,22,label_4,0.804411,0.712056,0.780137,0.773161,0.31887,0.804411


In [103]:
model_docs_lbl_similarities['prediction'] = model_docs_lbl_similarities['most_similar_label'].map({
    'label_0': 'Environmental', 'label_1': 'Infrastructure','label_2':'Education','label_3':'Zoning','label_4':'Safety'})
model_docs_lbl_similarities.head()

,doc_key,most_similar_label,highest_similarity_score,label_0,label_1,label_2,label_3,label_4,prediction
0,0,label_3,1.000000,0.256766,0.291182,0.269839,1.000000,0.323972,Zoning
1,1,label_4,0.529678,0.446110,0.511807,0.513902,0.448819,0.529678,Safety
2,2,label_3,0.495461,0.465151,0.453934,0.377555,0.495461,0.438389,Zoning
3,3,label_2,0.960554,0.692638,0.794938,0.960554,0.267855,0.734609,Education
4,4,label_1,0.470221,0.444783,0.470221,0.467993,0.442461,0.468819,Infrastructure


In [104]:
model_docs_lbl_similarities["prediction"].value_counts()

prediction
Education         10
Zoning             8
Safety             8
Infrastructure     7
Environmental      2
Name: count, dtype: int64

In [105]:
# evaluation_train = model_docs_lbl_similarities.merge(df, left_on='doc_key', right_on='doc_key')
y_true_train = y_train
y_pred_train = model_docs_lbl_similarities['prediction']
print('F1 score:',f1_score(y_true_train, y_pred_train, average='micro'))

F1 score: 0.5714285714285714


In [112]:
d = {'TRUE': y_train,'PRED': y_pred_train}
ff = pd.DataFrame(data=d)

ff.loc[ff.TRUE == ff.PRED, "score"] = 1
ff.loc[ff.TRUE != ff.PRED, "score"] = 0

### Accuracy is simple 

new_frame["score"].sum()/len(new_frame)

0.35294117647058826